In [1]:
import sqlite3
from pathlib import Path
 
REPO_PATH = Path("/Users/koen/workspace/agentic-syftbox")
SCREENPIPE_PATH = REPO_PATH / "data" / "screenpipe" / "db2.sqlite" 


In [2]:
HOME = Path.home()

In [3]:
MEETINGS_DB_PATH = HOME / ".meeting-notes-mcp" / "db.sqlite"
MEETINGS_DB_PATH.parent.mkdir(parents=True, exist_ok=True)

In [4]:
# MEETINGS_DB_PATH.unlink()

In [5]:
# Connect to the SQLite database
conn = sqlite3.connect(MEETINGS_DB_PATH)
conn.row_factory = sqlite3.Row

In [7]:
create_tables(conn)

# Create and query some meetings

In [10]:
# Connect to the SQLite database
screenpipe_conn = sqlite3.connect(SCREENPIPE_PATH)
screenpipe_conn.row_factory = sqlite3.Row

In [11]:
for row in screenpipe_conn.execute("""select * from audio_transcriptions""").fetchall():
    insert_transcription(conn, **row)

In [14]:
meetings = extract_meetings(conn)

In [15]:
for meeting in meetings:
    print(meeting.filename)
    print(meeting.show_text())
    print()

meeting_2025-06-03 14:30:48.452985+00:00.txt
So now that we defined the unique value proposition, I think we have a couple to dos couple to dos left that we want to define. The first one is to get screen Pipe running on my machine. Second to do is to make a recording of a meeting and share the SQLite database.
Then we need to create a ScreenPipe MCP server so we can query the transcription. And we need to try out the GitHub MCP server for creating tickets. And lastly, we need to try to run the end to end flow.

meeting_2025-06-05 13:00:36.357322+00:00.txt
 [BLANK_AUDIO]
 Oh, good job. Can you see it for sure? Yeah, okay.
 a boy chat box. Also when I was a Mark was one stater from Marvel travel. I was one. He used to have a fear of swine box with a fish back and...
 Alright!
 That's it. Okay.
 That's me at the top of the Supermope.
 Okay, I'll take one. Okay, just, okay. I'll shoot up.

meeting-2025-06-05 14:08:44.519520+00:00.txt
 (upbeat music)



In [16]:
for meeting in meetings:
    insert_meeting(conn, meeting.filename, meeting.chunks_ids)

In [19]:
for row in get_all_meeting_notes(conn):
    print(dict(row))

{'meeting_id': 1, 'filename': 'meeting_2025-06-03 14:30:48.452985+00:00.txt', 'start_date': '2025-06-03T14:30:48.452985+00:00', 'end_date': '2025-06-03T14:31:18.884843+00:00', 'full_text': 'So now that we defined the unique value proposition, I think we have a couple to dos couple to dos left that we want to define. The first one is to get screen Pipe running on my machine. Second to do is to make a recording of a meeting and share the SQLite database. Then we need to create a ScreenPipe MCP server so we can query the transcription. And we need to try out the GitHub MCP server for creating tickets. And lastly, we need to try to run the end to end flow.'}
{'meeting_id': 2, 'filename': 'meeting_2025-06-05 13:00:36.357322+00:00.txt', 'start_date': '2025-06-05T13:00:36.357322+00:00', 'end_date': '2025-06-05T14:08:44.519520+00:00', 'full_text': " [BLANK_AUDIO]  Oh, good job. Can you see it for sure? Yeah, okay.  a boy chat box. Also when I was a Mark was one stater from Marvel travel. I was

table meeting_meta

meeting_id | filename

table meeting_chunks

meeting_i | chunkid

```


```


Meetings CTE
| Meeting id | filename | full text | start date | end date


```
WITH meetings AS (
  SELECT
    mc.meeting_id,
    mm.filename,
    MIN(mc.date) AS start_date,
    MAX(mc.date) AS end_date,
    GROUP_CONCAT(mc.text, ' ') AS full_text
  FROM (
    SELECT * FROM meeting_chunks ORDER BY date, chunkid
  ) mc
  JOIN meeting_meta mm ON mc.meeting_id = mm.meeting_id
  GROUP BY mc.meeting_id
)
SELECT *
FROM meetings;
```

In [28]:
for chunks in meetings:
    date = chunks[0]["timestamp"]
    print(f"meeting {date}")
    for chunk in chunks:
        print(chunk["audio_chunk_id"])
    
    print()
    

meeting 2025-06-03T14:30:48.452985+00:00
1
2

meeting 2025-06-05T13:00:36.357322+00:00
3
4
4
5
5
6
7

meeting 2025-06-05T14:08:44.519520+00:00
8

